In [1]:
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
import warnings
import numpy as np
import pandas as pd
from v2.exceptions.data_exceptions import ValidationException
from v2.db.db_helper import get_max_inference_date
import datetime

from pprint import pprint

warnings.filterwarnings("ignore")

In [2]:
# import argparse
#
# parser = argparse.ArgumentParser()
# parser.add_argument('--save_to_db', help='Saving results to postgres', store='store_false')
# parser.add_argument('--save_to_file', help='Saving results to file', action='store_true')
# parser.add_argument('--show_shap', help='Showing SHAP results', action='store_true')
#
# arguments = parser.parse_args()

In [3]:
today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
last_month_with_last_day = last_month.strftime("%Y-%m-%d")
# inference_day = last_month.day
inference_month = last_month.month
inference_year = last_month.year

print(inference_month)
print(inference_year)

10
2023


In [4]:
#Validate data in DB - do we have such records in DB already?
def validate_dates(candidate_date_for_inference):
    max_date_from_db = get_max_inference_date()
    max_date_dt = datetime.datetime.strptime(max_date_from_db, '%Y-%m-%d')
    candidate_date_dt = datetime.datetime.strptime(candidate_date_for_inference, '%Y-%m-%d')
    if candidate_date_dt <= max_date_dt:
        raise ValidationException(f'Inference with date {candidate_date_for_inference} already exists in DB!')

def validate_data(candidate_date_for_inference):
    validate_dates(candidate_date_for_inference)

validate_data(last_month_with_last_day)

In [5]:
# READ DATA FROM THE DATA CATALOG
import json
with open('data_catalog.json', 'r') as file:
    data_catalog = json.load(file)

In [6]:
from v2.data_transformers.SequenceAttritionTrsTransformerNew import SequenceAttritionTrsTransformerNew

# 7/8/10 -> 31.#
inference_month = '10'
inference_year = '2023'
last_month_with_last_day = last_month.strftime("%Y-%m-%d")
report_date = '2023-10-31'# f'{last_month_with_last_day}'

input_paths = {
    'pred_data': f'<path_to_file>{inference_month}.{inference_year}.xlsx',
    'training': f'<path_to_file>.xlsx'
}

transformer = SequenceAttritionTrsTransformerNew(path_to_data=input_paths['pred_data'],
                                                 training_data_path=input_paths['training'],
                                                 predict=False,
                                                 cleanup=False)

data = transformer.prepare_data_for_attrition_prediction(min_date='01.01.2018')

2023-11-27 10:44:07,161 - v2.data_transformers.BaseTrsTransformer - INFO - Loading data from C:/Users/e-wdpk/Downloads/ddc/full_till_10.2023.xlsx
2023-11-27 10:44:24,116 - v2.data_transformers.BaseTrsTransformer - INFO - Employee column being generated...
2023-11-27 10:44:24,153 - v2.data_transformers.BaseTrsTransformer - INFO - Decoding polish characters...
2023-11-27 10:44:24,633 - v2.data_transformers.BaseTrsTransformer - INFO - Adding unique identifier...
2023-11-27 10:44:25,302 - v2.data_transformers.BaseTrsTransformer - INFO - Cleaning technology...
2023-11-27 10:44:25,313 - v2.data_transformers.BaseTrsTransformer - INFO - Data processed: shape (75334, 21), columns Index(['name', 'surname', 'status', 'division', 'grade', 'technology',
       'job family', 'start date', 'end date', 'office location',
       'contract type', 'client', 'current project', 'project status',
       'report_date', 'sap id', 'owner', 'line manager', 'unnamed: 0.1',
       'employee', 'unique_id'],
      

Using training data path: data/realized_trainings_till_2023-11-27.xlsx


2023-11-27 10:44:31,053 - v2.data_transformers.SequenceAttritionTrsTransformerNew - INFO - Removing columns... 


In [7]:
data.drop(['Zlecenie', 'Romania'], axis=1, inplace=True, errors='ignore')

In [8]:
data['current project'] = data['current project'].str.lower()
data['client'] = data['client'].str.lower()

In [9]:
#993 unique
current_data = data[data['left'] == 0]

In [10]:
def calc_month_of_work(data):
    # todo fix this calculation
    data['months_of_work'] = ((data['report_date_dt'] - data['start_date_dt']) / np.timedelta64(1, 'M'))
    data['months_of_work'] = data['months_of_work'].apply(lambda x: round(x))
    return data

def is_on_bench(x, y):
    # x = project, y = client
    x = x.lower()
    y = y.lower()
    if 'bench' in x or 'bench' in y:
        return 1
    elif 'internal' in y:
        return 1
    else:
        return 0

def avg_time_per_project(tenure, nth_project):
    return tenure / nth_project


def set_prob_period(x):
    if x < 4:
        return True
    else:
        return False


def get_tenure(x, y):
    months_of_empl = (y - x) / np.timedelta64(1, 'M')
    return months_of_empl / 12

def is_covid_employment(start_date):
    import datetime
    covid_start = datetime.datetime.strptime('20-03-2020', '%d-%m-%Y')
    covid_end = datetime.datetime.strptime('13-05-2022', '%d-%m-%Y')
    if covid_start < start_date < covid_end:
        return 1
    else:
        return 0

def get_time_to_event(date_column, event_column):
    return (risky[date_column] - risky.groupby(['unique_id', risky[event_column].eq(1).cumsum()])[date_column].transform("min"))/np.timedelta64(1, 'M')

def calc_months_till_now(data):
    data['months_till_now'] = ((data['max_date'] - data['report_date_dt']) / np.timedelta64(1, 'M'))
    data['months_till_now'] = data['months_till_now'].apply(lambda x: round(x))
    return data

In [11]:
current_data['tenure'] = current_data.apply(lambda x: get_tenure(x['start_date_dt'], x['report_date_dt']), axis=1)
current_data = calc_months_till_now(current_data)
current_data = calc_month_of_work(current_data)

In [12]:
current_data['client'] = current_data['client'].replace(' ', np.nan)
current_data['current project'] = current_data['current project'].replace(' ', np.nan)

In [13]:
current_data['client'] = current_data['client'].fillna('bench')
current_data['current project'] = current_data['current project'].fillna('bench')

In [14]:
current_data[current_data['current project'] == np.nan]

,name,surname,grade,technology,office location,client,current project,sap id,line manager,unnamed: 0.1,...,mapped_grade,left,office_location_prob_ratio,technology_prob_ratio,mapped_client_prob_ratio,SAP ID,has_training,tenure,months_till_now,months_of_work


In [15]:
risky = current_data.copy()

In [16]:
risky['current project'].astype(str)
risky['client'].astype(str)
#risky['current project'].fillna('No data', inplace=True)
#risky['client'].fillna('No data', inplace=True)
risky['is_on_bench'] = risky.apply(lambda x: is_on_bench(x['current project'], x['client']), axis=1)

In [17]:
#risky['is_covid_employment'] = risky['start_date_dt'].apply(is_covid_employment)
risky.sort_values(['unique_id', 'report_date_dt'], inplace=True)
risky['was_promoted'] = risky.groupby('unique_id').apply(
    lambda group: group['grade'] != group['grade'].shift(1)).tolist()
risky['was_promoted'] = risky['was_promoted'].map({True: 1, False: 0})
risky.loc[risky.groupby('unique_id').head(1).index, 'was_promoted'] = 1

risky['project_changed'] = risky.groupby('unique_id').apply(
    lambda group: group['current project'] != group['current project'].shift(1)).tolist()
risky['project_changed'] = risky['project_changed'].map({True: 1, False: 0})
risky.loc[risky.groupby('unique_id').head(1).index, 'project_changed'] = 0

risky['account_changed'] = risky.groupby('unique_id').apply(
    lambda group: group['client'] != group['client'].shift(1)).tolist()
risky['account_changed'] = risky['account_changed'].map({True: 1, False: 0})
risky.loc[risky.groupby('unique_id').head(1).index, 'account_changed'] = 0

In [18]:
risky["time_since_promo_in_months"] = get_time_to_event(date_column='report_date_dt', event_column='was_promoted')
risky["time_since_project_change_in_months"] = get_time_to_event(date_column='report_date_dt',
                                                                 event_column='project_changed')
risky["time_since_account_change_in_months"] = get_time_to_event(date_column='report_date_dt',
                                                                 event_column='account_changed')
risky["time_since_bench_in_months"] = get_time_to_event(date_column='report_date_dt', event_column='is_on_bench')
risky["time_since_training_in_months"] = get_time_to_event(date_column='report_date_dt', event_column='has_training')
risky.sort_values(['unique_id', 'report_date_dt'], inplace=True)
risky['bench_cumsum'] = risky.groupby(['unique_id'])['is_on_bench'].cumsum()
risky['bench_to_tenure'] = risky.apply(lambda x: (x['bench_cumsum'] / 12) / x['tenure'], axis=1)
risky['nth_project'] = risky.groupby(['unique_id'])[
                           'project_changed'].cumsum() + 1  # +1 beacuse it is reflecting the changes.
risky['avg_time_per_project'] = risky.apply(lambda x: avg_time_per_project(x['tenure'], x['nth_project']), axis=1)


risky['is_prob'] = risky['months_of_work'].apply(set_prob_period)
risky = risky[risky['is_prob'] == False]
risky.drop(['is_prob'], axis=1, inplace=True)

risky.drop(['months_before_termination', 'months_of_work', 'mapped_source', 'report_date', 'bench_cumsum',
            'left', 'diff', 'initial_grade', 'max_report_date_all', 'technology', 'start_date_dt',
             'Jjob family', 'source', 'status', 'division', 'technology', 'start date',
            'office location', 'contract type', 'current project', 'new_job_family', 'name', 'surname', 'last_project', 'last_client', 'mapped_project', 'mapped_client', 'has_training'
                                                                                       'is_on_bench', 'was_promoted',
            'project_changed', 'account_changed'], axis=1, inplace=True,
           errors='ignore')

In [19]:
#risky['is_covid_employment'] = risky['start_date_dt'].apply(is_covid_employment)

In [20]:
# MAPPING
all_predictions = True
if not all_predictions:
    account = '<account>'
    to_pred_df = risky[(risky['client'] == account) & (risky['report_date_dt'] == report_date)]
else:
    to_pred_df = risky[risky['report_date_dt'] == report_date]

In [21]:
# FILTERING THE L7
to_pred_df = to_pred_df[to_pred_df['grade'] != 'L7']

In [22]:
to_pred_df = to_pred_df.groupby('employee').last()
to_pred_df.reset_index(inplace=True)

In [23]:
idx = to_pred_df.index

mapping = pd.DataFrame({'employee_idx': idx,
                        'account': to_pred_df['client'],
                             'grade': to_pred_df['grade'],
                             'employee_unique_id': to_pred_df['unique_id'],
                             'employee': to_pred_df['employee']})

In [ ]:
mapping[mapping['employee'] == 'Witold Pawlak']

In [25]:
inference_date = to_pred_df['max_date'].head(1)
inference_date = inference_date[0].strftime('%Y-%m-%d')

In [26]:
to_pred_df.drop(['sampling', 'is_on_bench', 'has_training', 'unique_id', 'months_till_now', 'months_of_work', 'mapped_source', 'report_date', 'bench_cumsum',
            'stay/left', 'diff', 'initial_grade', 'max_report_date_all', 'max_date', 'technology', 'start_date_dt',
            'report_date_dt', 'employee', 'job family', 'source', 'status', 'division', 'technology', 'start date',
            'office location', 'contract type', 'client', 'current project', 'new_job_family', 'name','grade', 'surname', 'last_project', 'last_client', 'other', 'sap id'], axis=1, inplace=True,
           errors='ignore')

In [27]:
import joblib

desired_artifact_id = '8ff1eaa2-0887-4605-b20d-4f1bade5cf85'

def load_desired_artifacts(artifact_id: str):
    with open(f'artifacts/{artifact_id}/models/model.joblib', 'rb') as file:
        model = joblib.load(file)

    training_data = pd.read_csv(f'artifacts/{artifact_id}/data/training_data.csv', sep=';')
    return model, training_data

model, training_data = load_desired_artifacts(desired_artifact_id)

In [28]:
training_columns = training_data.columns
to_pred_df = to_pred_df[training_columns]

In [29]:
assert to_pred_df.shape[1] == training_data.shape[1]

In [30]:
predictions = model.predict(to_pred_df)

In [31]:
probalities = model.predict_proba(to_pred_df)[:, 1]

In [32]:
def get_risk_bucket(x):
    if x <= 0.5:
        return 'no risk'
    elif x <= 0.7:
        return 'low risk'
    elif x <= 0.8:
        return 'moderate risk'
    elif x <= 0.9:
        return 'high risk'
    else:
        return 'very high risk'

In [33]:
results = pd.DataFrame({'employee': mapping['employee'],
                        'account': mapping['account'],
                        'grade': mapping['grade'],
                        'inference_date': inference_date,
                        'at_risk': predictions,
                        'probability': probalities})

In [34]:
results['risk_bucket'] = results['probability'].apply(get_risk_bucket)

In [35]:
# from model_explainers.lime_explainer import setup_explainer_on_train_data, prepare_contributors_for_prediction
#
# lime_explainer = setup_explainer_on_train_data(training_data)
# contributors = pd.concat(prepare_contributors_for_prediction(to_pred_df, model, explainer=lime_explainer), axis=0, ignore_index=True)
# contributors

In [36]:
def get_dynamic_columns_mapping(columns) -> dict:
    mapping = {}

    for i in range(len(columns)):
        mapping[i] = columns[i]

    return mapping

In [37]:
import shap
#shap.initjs()
from tqdm import tqdm

#shap_values = explainer.shap_values(to_pred_df)

columns_mapper = get_dynamic_columns_mapping(to_pred_df.columns)
explainer = shap.TreeExplainer(model)

def get_contributors_with_shap(explainer, prediction_data):
    all_contributors = []
    n_predictions = prediction_data.shape[0]
    shap_values = explainer(to_pred_df)
    for i in tqdm(range(n_predictions)):
        impact_values = pd.DataFrame(shap_values[i].values)
        impact_df = impact_values.T
        impact_df = impact_df.rename(columns=columns_mapper)
        all_contributors.append(impact_df)

    return all_contributors

In [38]:
contributors_shap = pd.concat(get_contributors_with_shap(explainer, to_pred_df), axis=0, ignore_index=True)

100%|██████████| 1094/1094 [00:01<00:00, 672.40it/s]


In [39]:
pred_and_impact_df = pd.concat([results, contributors_shap], axis=1)

In [40]:
feat_importances = pd.Series(model.feature_importances_, index=training_data.columns)
feat_importances.sort_values(ascending=False).head()

time_since_bench_in_months    0.110895
time_since_promo_in_months    0.110050
avg_time_per_project          0.108647
nth_project                   0.105253
mapped_client_prob_ratio      0.102784
dtype: float32

In [41]:
# import shap
# shap.initjs()
# explainer = shap.TreeExplainer(model)
# #shap_values = explainer.shap_values(to_pred_df)
# shap_values = explainer(to_pred_df)

In [42]:
#shap.summary_plot(shap_values) # wez ten wykres, jest super
#shap.plots.beeswarm(shap_values) # wez ten wykres, jest super
#shap.waterfall_plot(shap_values[1108])

In [43]:
import os
import pandas as pd

def check_if_file_exists(path_to_file):
    return os.path.exists(path_to_file)

save_to_file = False

if save_to_file:
    file_name = ''

    if not all_predictions:
        file_name = f'{account}_predictions_{report_date}.xlsx'
    else:
        file_name = f'all_predictions_grade_{report_date}.xlsx'

    if check_if_file_exists(file_name):
        print(f'File {file_name} exists, opening...')
        existing_predictions_df = pd.read_excel(file_name)
        print(f'Pre shape: {existing_predictions_df.shape}')
        print(f'Pre shape results: {results.shape}')
        all = pd.concat([existing_predictions_df, results], axis=0)
        print(f'Post shape: {all.shape}')
        all.to_excel(file_name)
    else:
        results.to_excel(file_name)

In [44]:
from v2.db.db_helper import connect_to_db
engine = connect_to_db()
schema = 'public'
table = 'predictions_with_explainers'
pred_and_impact_df.to_sql(f'{table}', con=engine, if_exists='append', schema=f'{schema}')